In [6]:
import numpy as np
import torch
import torch.nn as nn

import os

os.environ["CUDA_VISIBLE_DEVICES"] = '1'


def convert_bytes(size):
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            return "%3.2f %s" % (size, x)
        size /= 1024.0
    return size

def pmem(txt=''):
    print(txt, convert_bytes(torch.cuda.memory_allocated()), convert_bytes(torch.cuda.max_memory_allocated()))

In [ ]:
torch.cuda.memory_allocated()

### Transformer memory

In [7]:
pmem('init')

bs = 1
dim = (bs,512,512,256)
ps = (16,16,16)
d_model = np.prod(ps)
n_layers = 1
rs_dim = [bs] + [int(i/j) for i,j in zip(dim[1:], ps)]


x = torch.from_numpy(np.random.rand(dim)).float().cuda()
pmem('input')

tr = nn.TransformerDecoderLayer(d_model=d_model, n_head=1)
mod = nn.TransformerEncoder(tr, num_layers=n_layers).float().cuda()
mod.train()
pmem('model')

x = torch.reshape(x, rs_dim)
pmem('reshape')

y = tr(x)
pmem('forward')


RuntimeError: cuda runtime error (3) : initialization error at /pytorch/aten/src/THC/THCGeneral.cpp:47